In [33]:
import Bio
from Bio import SeqIO
import csv


set1_name = 'trees_ML/algn/notch4/re_new/set1.fa'
set2_name = 'trees_ML/algn/notch4/re_new/notch4.fa'
var_file = 'variants/notch4_table'

In [34]:
set1_msa={}

for line in SeqIO.parse(set1_name,"fasta"):
    set1_msa[line.id] = str(line.seq)


In [35]:
set2_msa={}

for line in SeqIO.parse(set2_name,"fasta"):
    set2_msa[line.id] = str(line.seq)

In [36]:
set_diff_msa={}

for i in set2_msa:
    if i not in set1_msa.keys():
        set_diff_msa[i] = set2_msa[i]

In [37]:
clinvar=[]
with open(var_file) as f:
    for line in f:
        clinvar.append(str(line))       
clean_clinvar=[]
for i in clinvar:
    new=i.strip("\n")
    clean_clinvar.append(new.split(" "))
#clean_clinvar

In [38]:
header=['Input', 'Prediction', 'Conservative', 'Substitutions_Set1'\
    , "N_Substitutions_Set1", 'Substitutions_Set2','N_Substitutions_Set2/Set1']

classes = {'p1':[],'p2':[],'p3':[],'p4':[],'p5':[],'p6':[],'p7':[],'p8':[]}

with open('n4_saver_test.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)

    for x in clean_clinvar: 
        # Positions
        pos=int(x[1])-1 # position of interest in alligned sequences
        AA1=x[-1]

        # Human positions

# NP_000426.2_neurogenic_locus_notch_homolog_protein_3_precursor_[Homo_sapiens]= n3
# NP_060087.3 = n1
# NP_077719.2 = n2
# NP_004548.3 = n4

        human=set1_msa['NP_004548.3'] # human sequence
        human_pos=set1_msa['NP_004548.3'][pos]

        # Gaps and X-amino acids
        gap="-"
        X="X"

        # Counters
        AA0_count = 0 #counts how many amino acids identical to human are present in pos
        AA1_set1_count = 0 #counts how many AA1 are present in pos in set1
        AA1_set2_count = 0
        AA1_set_diff_count = 0 #counts how many AA1 are present in pos in set2-set1
        X_count=0 #?
        gap_count=0 #?

        # Lists of amino acids
        set1_sub_list=[] # will collect amino acids subsititions in set1
        set2_sub_list=[] # will collect amino acids subsititions in set2
        set_diff_sub_list=[]

        # Human lists
        human_list=[human_pos,gap,X] 

        for seq in list(set1_msa.values())[1:]: # start from 1, because first(0) sequence is human

            if seq[pos] in human_list:
                AA0_count += 1
            elif seq[pos] == AA1:
                AA1_set1_count += 1
                set1_sub_list.append(seq[pos])
            else:
                set1_sub_list.append(seq[pos])
        
        for seq2 in list(set2_msa.values())[1:]: # checking AA1 in set2
            
            if seq2[pos] == AA1:
                AA1_set2_count += 1
                set2_sub_list.append(seq2[pos])
        
        for seq3 in list(set_diff_msa.values()): #counting AA1 in set2-set1
            
            if seq3[pos] == AA1:
                AA1_set_diff_count += 1
                set_diff_sub_list.append(seq3[pos])
                
        
        # Editing the lists
        set1_sub_list = list(set(set1_sub_list))
        set1_sub_list = (",".join(set1_sub_list))
        set1_sub_list = set1_sub_list.replace(","," ")
        
        set2_sub_list = list(set(set2_sub_list))
        set2_sub_list = (",".join(set2_sub_list))
        set2_sub_list = set2_sub_list.replace(","," ")
        
        x = (",".join(x))
        x = x.replace(","," ")
        
                    
        if AA0_count == len(set1_msa)-1: # AA0 fully conserved in set1 (first sequence is human)
            classes['p1'].append(x)
            data = (x,'Damaging',AA0_count, set1_sub_list, AA1_set1_count,\
                set2_sub_list, AA1_set_diff_count)
            writer.writerow(data)
        elif AA1_set1_count > 1: # Set1 has AA1 with > 1 events
            data=(x,'Benign',AA0_count, set1_sub_list, AA1_set1_count,\
                set2_sub_list, AA1_set_diff_count)
            writer.writerow(data)
            classes['p2'].append(x)
        elif AA1_set1_count == 1 and (AA1 in set_diff_sub_list): #Set1 has single event of AA1 and AA1 is alse seen in set2-set1
            data=(x,'Benign',AA0_count, set1_sub_list, AA1_set1_count,\
                set2_sub_list, AA1_set_diff_count)
            writer.writerow(data)
            classes['p3'].append(x)
        elif AA1 not in set2_sub_list: # AA1 not seen in set2
            data=(x,'Damaging',AA0_count, set1_sub_list, AA1_set1_count,\
                set2_sub_list, AA1_set_diff_count)
            writer.writerow(data)
            classes['p4'].append(x)
        elif len(set1_sub_list) < 3: # Position not moderately variable in set1 
            data=(x,'Damaging',AA0_count, set1_sub_list, AA1_set1_count,\
                set2_sub_list, AA1_set_diff_count)
            writer.writerow(data)
            classes['p5'].append(x)
        elif len(set1_sub_list) in range(3,6) and (AA1_set_diff_count >1): # Position moderately variable in set1 and set2-set1 has AA1 with >1 event
            data=(x,'Benign',AA0_count, set1_sub_list, AA1_set1_count,\
                set2_sub_list, AA1_set_diff_count)
            writer.writerow(data)
            classes['p6'].append(x)
        elif len(set1_sub_list) > 5: # Position hyper-variable in set 1
            data=(x,'Benign',AA0_count, set1_sub_list, AA1_set1_count,\
                set2_sub_list, AA1_set_diff_count)
            writer.writerow(data)
            classes['p7'].append(x)
        else:
            data=(x,'Damaging',AA0_count, set1_sub_list, AA1_set1_count,\
                set2_sub_list, AA1_set_diff_count)
            writer.writerow(data)
            classes['p8'].append(x)

In [39]:
classes['p1']

['NP_004548.3 1213 G R', 'NP_004548.3 1099 G R', 'NP_004548.3 815 C G']

In [41]:
import pandas as pd
df = pd.read_csv('n4_saver_test.csv')
df.Prediction.value_counts()

Benign      7
Damaging    3
Name: Prediction, dtype: int64